# Beating the Game Show

This is a solution to the Riddler Classic problem found here: https://fivethirtyeight.com/features/can-you-beat-the-game-show/

I approached this problem in a pretty lazy, brute-force, non-theoretical way, and then used some not-completely-airtight reasoning to arrive at what I believe is the correct solution to the "extra credit" question.

Ok. We got four boxes and four names:

In [1]:
boxes = [1, 2, 3, 4]
names = ['A', 'B', 'C', 'D']

I used the itertools module to make three lists:
1. All the ways the names can be arranged in the boxes
2. All the ways a single player can pick two of the four boxes
3. All the ways that all four players can pick their boxes

In [2]:
import itertools

name_orders = list(itertools.permutations(names))
individual_strategies = list(itertools.combinations(boxes, 2))
group_strategies = list(itertools.product(individual_strategies, repeat=4))

There are 1296 possible group strategies:

In [3]:
len(group_strategies)

1296

Let's take a look at an example of what a group strategy looks like.

In [4]:
group_strategies[800]

((2, 3), (2, 4), (1, 3), (1, 4))

This means that the 800th strategy in our list of 1296 strategies is:
- A picks boxes 2 and 3
- B picks boxes 2 and 4
- C picks boxes 1 and 3
- D picks boxes 1 and 4

Moving along... here's a function that tests a strategy by seeing how it does against all possible arrangements of names in the boxes:

In [5]:
def eval_strategy(s):
    '''Calculate probability that strategy s will win.'''
    wins = 0
    for order in name_orders:
        wins += sum([order.index(names[i])+1 in s[i] for i in range(4)]) == 4
    return wins / float(len(name_orders))

Let's evaluate all of our strategies. What's the best we can do?

In [6]:
results = [eval_strategy(s) for s in group_strategies]
max(results)

0.16666666666666666

There are some strategies that give us a 16.66 percent chance of winning.
Let's take a look at all the ones that do that.

In [7]:
[group_strategies[i] for i in range(1296) if results[i] == max(results)]

[((1, 2), (1, 2), (3, 4), (3, 4)),
 ((1, 2), (3, 4), (1, 2), (3, 4)),
 ((1, 2), (3, 4), (3, 4), (1, 2)),
 ((1, 3), (1, 3), (2, 4), (2, 4)),
 ((1, 3), (2, 4), (1, 3), (2, 4)),
 ((1, 3), (2, 4), (2, 4), (1, 3)),
 ((1, 4), (1, 4), (2, 3), (2, 3)),
 ((1, 4), (2, 3), (1, 4), (2, 3)),
 ((1, 4), (2, 3), (2, 3), (1, 4)),
 ((2, 3), (1, 4), (1, 4), (2, 3)),
 ((2, 3), (1, 4), (2, 3), (1, 4)),
 ((2, 3), (2, 3), (1, 4), (1, 4)),
 ((2, 4), (1, 3), (1, 3), (2, 4)),
 ((2, 4), (1, 3), (2, 4), (1, 3)),
 ((2, 4), (2, 4), (1, 3), (1, 3)),
 ((3, 4), (1, 2), (1, 2), (3, 4)),
 ((3, 4), (1, 2), (3, 4), (1, 2)),
 ((3, 4), (3, 4), (1, 2), (1, 2))]

So these are all the optimal strategies. Note that all of these strategies follow the same general plan: <b>divide all the players up evenly into two groups, have every member of the first group look in the same set of boxes, and have every member of the second group look in the set of boxes that the first group didn't look in.</b>

Does this strategy generalize to situations with more players and boxes? Yeah! Seems like it! I used the lazy, brute-force method to analyze the game when there are six players and six boxes and found that the "half the players choose one set, half the players choose the other set" method is still optimal.

So let's assume that it's optimal for the 100 player, 100 boxes game. Here's our strategy: 
- The first 50 players look for their names in boxes {1, 2, ... , 50}
- The next 50 players look for their names in boxes {51, 52, ..., 100}

The probability of winning with this strategy is the probability of drawing (without replacement) 50 consecutive red balls from a bag that is originally filled with 50 red balls and 50 blue balls, i.e., (50/100) × (49/99) × (48/98) × ... × (1/51).

In other words, it's this number:

In [8]:
reduce(lambda x, y: x*y, range(1, 51))/float(reduce(lambda x, y: x*y, range(51, 101)))

9.911653021418339e-30

And our strategy gives us a chance of winning that's 12.5 times greater than the naive strategy:

In [9]:
9.911653021418339e-30 / float(0.5**100)

12.5645129018549